In [4]:
%pylab inline
import numpy as np
import csv


Populating the interactive namespace from numpy and matplotlib


In [5]:
def preprocess(file_name):
    results = []
    with open(file_name) as csvfile:
        wine_reader = csv.reader(csvfile, dialect="excel", delimiter=';')
        for row in wine_reader:
            results.append(row)
    name_features = results[0]
    #print(name_features)
    parsed_results = []
    for row in results[1:]:
        new_row = []
        for num in row:
            new_row.append(float(num))
        parsed_results.append(new_row)
    parsed_results = np.array(parsed_results)
    n_instances = parsed_results.shape[0]
    n_features = parsed_results.shape[1] - 1
    labels = parsed_results[:, n_features]
    instances = parsed_results[:, : n_features]
    return instances, labels
white_instances, white_labels = preprocess('winequality-white.csv')
red_instances, red_labels = preprocess('winequality-red.csv')
transformed_white_labels = np.array([0 if num <6 else 1 for num in white_labels])
transformed_red_labels = np.array([0 if num <6 else 1 for num in red_labels])


In [24]:
from sklearn.model_selection import train_test_split

#Additional imports down here:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV

In [25]:
X_train_white, X_test_white, y_train_white, y_test_white = train_test_split(white_instances, transformed_white_labels)

In [26]:
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(red_instances, transformed_red_labels)

In [27]:
scaler = StandardScaler()
scaler.fit(X_train_white)
scaler.fit(X_test_white)
normalised_X_train_white = scaler.transform(X_train_white)
normalised_X_test_white = scaler.transform(X_test_white)

In [28]:
scaler.fit(X_train_red)
scaler.fit(X_test_red)
normalised_X_train_red = scaler.transform(X_train_red)
normalised_X_test_red = scaler.transform(X_test_red)

In [29]:
svm_clf = svm.SVC()
svm_clf.fit(normalised_X_train_red,y_train_red)
def report_accuracy(moment, clf_name, clf, x_train, y_train):
    cv_error = cross_val_score(clf, x_train, y_train, scoring="accuracy", cv=4)
    print(moment + " customisation, average accuracy of " + clf_name + " : ", cv_error.mean(), " standard deviation: ", np.std(cv_error))

report_accuracy("Before", "SVM_SVC", svm_clf, normalised_X_train_red, y_train_red) 

Before customisation, average accuracy of SVM_SVC :  0.769813266444  standard deviation:  0.036709873706
